In [206]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import re 
from geopy.distance import geodesic
from datetime import datetime, timedelta, time, date
import matplotlib.pyplot as plt
# Example 5. FAM (Fleet Assignment Model)

In [292]:
# Define the inputs
# Define the inputs

# L: set of flights
flights_df = pd.read_excel('Example_Data_Fam.xlsx', sheet_name='Flight').set_index('Flight no.')
flights = flights_df.to_dict(orient='index')

# K: set of aircraft types
aircraft_df = pd.read_excel('Example_Data_Fam.xlsx', sheet_name='Aircraft').set_index('AC Type')
aircraft = aircraft_df.to_dict(orient='index')

for k in aircraft:
    aircraft[k]['TAT'] = timedelta(minutes=aircraft[k]['TAT'])


flight_list = list(flights.keys())
ac_list = list(aircraft.keys())

In [295]:
flights_df['Overnight'] = flights_df.apply(lambda row: (row['Arrival'] < row['Departure']), axis=1)

In [208]:
flight_distance = {     # Distance [km]
    'PDL': {'PDL': 0, 'LIS': 1461, 'OPO': 1536, 'FNC': 975, 'YTO': 4545, 'BOS': 3888},
    'LIS': {'PDL': 1461, 'LIS': 0, 'OPO': 336, 'FNC': 973, 'YTO': 5790, 'BOS': 5177},
    'OPO': {'PDL': 1536, 'LIS': 336, 'OPO': 0, 'FNC': 1244, 'YTO': 5671, 'BOS': 5081},
    'FNC': {'PDL': 975, 'LIS': 973, 'OPO': 1244, 'FNC': 0, 'YTO': 5515, 'BOS': 4851},
    'YTO': {'PDL': 4545, 'LIS': 5790, 'OPO': 5671, 'FNC': 5515, 'YTO': 0, 'BOS': 691},
    'BOS': {'PDL': 3888, 'LIS': 5177, 'OPO': 5081, 'FNC': 4851, 'YTO': 691, 'BOS': 0}
}

In [209]:
for l in flights:
    flights[l]['Distance'] = flight_distance[flights[l]['From']][flights[l]['To']]

In [210]:
current_time = time(0, 0)
airports  = list(set([flights[l]['From'] for l in flights] + [flights[l]['To'] for l in flights]))

In [234]:
import pandas as pd

# Create an empty list to store the data
data = []

misc_date = date(1,1,1)
# Iterate over airports, aircraft types, and flights
for n in airports:
    for k in ac_list:
        for l in flight_list:
            if flights[l]['From'] == n:
                data.append([k, n, l, flights[l]['Departure'], flights[l]['Distance'], 'Departure'])
            if flights[l]['To'] == n:
                data.append([k, n, l, flights[l]['Arrival'], flights[l]['Distance'], 'Arrival'])

# Create a dataframe with the data
events = pd.DataFrame(data, columns=['AC Type','Airport', 'Flight N', 'Time', 'Distance', 'D_A'])

# Add the TAT to the arrival times
events['Time'] = events.apply(lambda row: (datetime.combine(misc_date,row['Time']) + aircraft[row['AC Type']]['TAT']).time() if row['D_A'] == 'Arrival' else row['Time'], axis=1)

events.sort_values(by=['AC Type', 'Airport', 'Time'], inplace=True)
# Remove the flights that are not feasible because of range of aircraft
events = events[~events.apply(lambda row: (row['Distance'] > aircraft[row['AC Type']]['Range']), axis=1)]

# Reset the numbering of the events
events.reset_index(drop=True, inplace=True)

In [212]:
import plotly.graph_objects as go

for k in ac_list:
    df = events[events['AC Type'] == k].sort_values(by=['Time'])
    fig = go.Figure()
    
    # Set marker color based on 'D_A' column
    marker_color = ['red' if d_a == 'Departure' else 'blue' for d_a in df['D_A']]
    
    fig.add_trace(go.Scatter(x=df['Time'], y=df['Airport'], mode='markers', name='markers', marker=dict(color=marker_color)))

    fig.update_layout(
        title="Ground Arcs for AC Type " + k,
        xaxis_title="Time",
        yaxis_title="Airport",
    )
    fig.show()


In [213]:
df_test = events[(events['AC Type'] == 'A310') & (events['Airport'] == 'PDL')].sort_values(by=['Time'])

ground_arcs = pd.DataFrame(columns=['AC Type', 'Airport', 'Start Time', 'End Time'])
for i in range(len(df_test)):
    if i == 0:
        ground_arcs = pd.concat([ground_arcs, pd.DataFrame({'AC Type': 'A310', 'Airport': 'PDL', 'Start Time': [df_test.iloc[-1]['Time']], 'End Time': [df_test.iloc[i]['Time']]})], ignore_index=True)
    else:
        ground_arcs = pd.concat([ground_arcs, pd.DataFrame({'AC Type': 'A310', 'Airport': 'PDL', 'Start Time': [df_test.iloc[i-1]['Time']], 'End Time': [df_test.iloc[i]['Time']]})], ignore_index=True)

ground_arcs

,AC Type,Airport,Start Time,End Time
0,A310,PDL,22:05:00,08:40:00
1,A310,PDL,08:40:00,09:00:00
2,A310,PDL,09:00:00,09:25:00
3,A310,PDL,09:25:00,09:25:00
4,A310,PDL,09:25:00,09:35:00
5,A310,PDL,09:35:00,10:35:00
6,A310,PDL,10:35:00,11:10:00
7,A310,PDL,11:10:00,11:20:00
8,A310,PDL,11:20:00,15:15:00
9,A310,PDL,15:15:00,16:00:00


In [260]:
# For each airport and aircraft type i need to create a loop of ground arcs, each starting from the last event and ending at the next event, if there is no more events then the last ground arc is the overnight arc and it ends at the first event of the next day (first event of the next day is the first event of the same airport and aircraft type)

ground_arcs = pd.DataFrame(columns=['AC Type', 'Airport', 'Start Time', 'End Time'])

for k in ac_list:
    for n in airports:
        df = events[(events['AC Type'] == k) & (events['Airport'] == n)].sort_values(by=['Time'])
        for i in range(len(df)):
            if i == 0:
                ground_arcs = pd.concat([ground_arcs, pd.DataFrame({'AC Type': k, 'Airport': n, 'Start Time': [df.iloc[-1]['Time']], 'End Time': [df.iloc[i]['Time']]})], ignore_index=True)
            else:
                ground_arcs = pd.concat([ground_arcs, pd.DataFrame({'AC Type': k, 'Airport': n, 'Start Time': [df.iloc[i-1]['Time']], 'End Time': [df.iloc[i]['Time']]})], ignore_index=True)

ground_arcs.sort_values(by=['AC Type', 'Airport'], inplace=True)
# Drop rows if start time and end time are the same
ground_arcs = ground_arcs[~ground_arcs.apply(lambda row: (row['Start Time'] == row['End Time']), axis=1)]
ground_arcs['Arc ID'] = ground_arcs.groupby(['AC Type']).cumcount()
ground_arcs['Overnight'] = ground_arcs.apply(lambda row: (row['End Time'] < row['Start Time']), axis=1)

In [270]:
# Create a nodes_df
nodes_df = ground_arcs[['AC Type', 'Airport', 'Start Time']].rename(columns={'Start Time': 'Time'})

# Add a count number for each row group by AC Type and Airport
nodes_df['Node ID'] = nodes_df.groupby(['AC Type']).cumcount()

# make a dictionary with the ac type as main key and the airport as secondary key with the node as tertiary key and the time as value
nodes = {}
for k in ac_list:
    nodes[k] = {}
    for n in airports:
        nodes[k][n] = {}
        for i in nodes_df[(nodes_df['AC Type'] == k) & (nodes_df['Airport'] == n)]['Node ID']:
            nodes[k][n][i] = {'Time': nodes_df[(nodes_df['AC Type'] == k) & 
                                               (nodes_df['Airport'] == n) & 
                                               (nodes_df['Node ID'] == i)]['Time'].values[0]}
            

In [272]:
# Add node id to the events
events.merge(nodes_df, how='left', on=['AC Type', 'Airport', 'Time']).sort_values(by=['AC Type', 'Airport'])

# Add a dictionary call departures and another one arrivals to the nodes dictionary at k,n,i+1 with 
# the events that have the same ac type, airport and time as the node
for k in ac_list:
    for n in airports:
        for i in nodes_df[(nodes_df['AC Type'] == k) & (nodes_df['Airport'] == n)]['Node ID']:
            nodes[k][n][i]['Departures'] = list(events[(events['AC Type'] == k) & (events['Airport'] == n) & (events['Time'] == nodes[k][n][i]['Time']) & (events['D_A'] == 'Departure')]['Flight N'])
            nodes[k][n][i]['Arrivals'] = list(events[(events['AC Type'] == k) & (events['Airport'] == n) & (events['Time'] == nodes[k][n][i]['Time']) & (events['D_A'] == 'Arrival')]['Flight N'])


In [251]:
ground_arcs[ground_arcs['AC Type'] == 'A310']

,AC Type,Airport,Start Time,End Time,Arc ID,Overnight
0,A310,BOS,22:25:00,03:15:00,0,True
1,A310,BOS,03:15:00,22:25:00,1,False
2,A310,FNC,17:05:00,13:25:00,2,True
3,A310,FNC,13:25:00,17:05:00,3,False
26,A310,LIS,18:50:00,00:50:00,4,True
27,A310,LIS,00:50:00,06:30:00,5,False
28,A310,LIS,06:30:00,08:05:00,6,False
29,A310,LIS,08:05:00,11:00:00,7,False
30,A310,LIS,11:00:00,12:10:00,8,False
31,A310,LIS,12:10:00,14:05:00,9,False


In [291]:
# n+: ground arcs originating at any node n (start time)
# n-: ground arcs ending at any node n (end time)
n_plus = ground_arcs[['Airport', 'AC Type', 'Start Time', 'Arc ID']].rename(columns={'Start Time': 'Time'})
n_minus = ground_arcs[['Airport', 'AC Type', 'End Time', 'Arc ID']].rename(columns={'End Time': 'Time'})

# Add a dictionary call n+ and another one n- to the nodes dictionary at k,n,i+1 with 
# the events that have the same ac type, airport and time as the node
for k in ac_list:
    for n in airports:
        for i in nodes_df[(nodes_df['AC Type'] == k) & (nodes_df['Airport'] == n)]['Node ID']:
            nodes[k][n][i]['n+'] = list(n_plus[(n_plus['AC Type'] == k) & (n_plus['Airport'] == n) & (n_plus['Time'] == nodes[k][n][i]['Time'])]['Arc ID'])
            nodes[k][n][i]['n-'] = list(n_minus[(n_minus['AC Type'] == k) & (n_minus['Airport'] == n) & (n_minus['Time'] == nodes[k][n][i]['Time'])]['Arc ID'])

In [132]:
# Create an outbound and inbound dictionary
outbound = {}
inbound = {}
unfeasible = {}
for k in ac_list:
    outbound[k] = {}
    inbound[k] = {}
    for n in airports:
        outbound[k][n] = []
        inbound[k][n] = []
        for l in flight_list:
            if flights[l]['Distance'] <= aircraft[k]['Range']:
                if flights[l]['From'] == n:
                    outbound[k][n].append(l)
                if flights[l]['To'] == n:
                    inbound[k][n].append(l)
            else:
                unfeasible[l] = True

inbound

{'A310': {'BOS': ['S4221'],
  'FNC': ['S4160'],
  'OPO': ['S4320', 'S4212'],
  'PDL': ['S4121',
   'S4229',
   'S4125',
   'S4129',
   'S4321',
   'S4213',
   'S4161',
   'S4223',
   'S4230'],
  'LIS': ['S4120', 'S4220', 'S4124', 'S4128'],
  'YTO': ['S4222']},
 'A320': {'BOS': ['S4221'],
  'FNC': ['S4160'],
  'OPO': ['S4320', 'S4212'],
  'PDL': ['S4121',
   'S4229',
   'S4125',
   'S4129',
   'S4321',
   'S4213',
   'S4161',
   'S4230'],
  'LIS': ['S4120', 'S4220', 'S4124', 'S4128'],
  'YTO': []}}

In [297]:
overnight_arcs = ground_arcs[ground_arcs['Overnight'] == True][['AC Type', 'Airport', 'Start Time', 'End Time']].rename(columns={'Start Time': 'Time'})
flights_df
#overnight_flights = flights_df[flights_df['Overnight'] == True][['AC Type', 'From', 'To', 'Departure', 'Arrival']].rename(columns={'Departure': 'Time'})

,From,To,Departure,Arrival,Demand,Overnight
Flight no.,,,,,,
S4120,PDL,LIS,09:25:00,11:30:00,195,False
S4220,PDL,LIS,11:20:00,13:25:00,167,False
S4124,PDL,LIS,16:05:00,18:10:00,151,False
S4128,PDL,LIS,22:05:00,00:10:00,184,True
S4320,PDL,OPO,09:35:00,11:45:00,166,False
S4212,PDL,OPO,19:00:00,21:10:00,134,False
S4160,PDL,FNC,10:35:00,12:45:00,155,False
S4222,PDL,YTO,17:10:00,23:45:00,214,False
S4221,PDL,BOS,16:00:00,21:45:00,198,False


In [287]:
# Define the model
m = gp.Model('FAM')

# Define the decision variables
# f[i,k] 1 if flight arc i is assigned to aircraft type k, 0 otherwise
f = {}
for i in flight_list:
    for k in ac_list:
        f[i, k] = m.addVar(vtype=GRB.BINARY, name='f_' + str(i) + '_' + str(k))

# y_ak = number of aircraft of type k on the ground arc a (integer with lowerbound 0)
y = {}
for k in ac_list:
    for a in range(len(ground_arcs[(ground_arcs['AC Type'] == k)])):
        y[a, k] = m.addVar(vtype=GRB.INTEGER, name='y_' + str(a) + '_' + str(k))

m.update()

# Define the objective function
m.setObjective(gp.quicksum(aircraft[k]['CASK'] *        # Cost per Available Seat Kilometer
                           aircraft[k]['Seats'] *       # Number of seats
                           flights[i]['Distance'] *     # Distance
                           f[i,k]                       # Binary variable
                           for i in flight_list for k in ac_list), GRB.MINIMIZE)


# Define the constraints
# Constraint 1: Each flight is assigned to exactly one aircraft type
m.addConstrs((gp.quicksum(f[i,k] for k in ac_list) == 1 for i in flight_list), name='one_ac')

# Constraint 2: The number of AC arriving = AC departing, for each type at each node
# y_n+_k + sum(f_i,k) = y_n-_k + sum(f_i,k)
for k in ac_list:
    for n in airports:
        for i in nodes[k][n]:
            m.addConstr((y[nodes[k][n][i]['n+'][0], k] + gp.quicksum(f[i,k] for i in nodes[k][n][i]['Arrivals']) == y[nodes[k][n][i]['n-'][0], k] + gp.quicksum(f[i,k] for i in nodes[k][n][i]['Departures'])), name='balance_' + str(i) + '_' + str(k) + '_' + str(n))

# Constraint 3: The number 
m.update()

In [290]:
for i in nodes['A310']['OPO']:
    print(y[nodes['A310']['OPO'][i]['n+'][0], 'A310'].varName)
    for w in nodes['A310']['OPO'][i]['Arrivals']:
        print(f[w, 'A310'].varName)
    print(y[nodes['A310']['OPO'][i]['n-'][0], 'A310'].varName)
    for w in nodes['A310']['OPO'][i]['Departures']:
        print(f[w, 'A310'].varName)
    print('')

y_11_A310
f_S4212_A310
y_14_A310

y_12_A310
f_S4320_A310
y_11_A310

y_13_A310
y_12_A310
f_S4321_A310

y_14_A310
y_13_A310
f_S4213_A310



In [247]:
for v in m.getVars():
    if v.varName[0] == 'y':
        print(v.varName)

y_0_A310
y_1_A310
y_2_A310
y_3_A310
y_4_A310
y_5_A310
y_6_A310
y_7_A310
y_8_A310
y_9_A310
y_10_A310
y_11_A310
y_12_A310
y_13_A310
y_14_A310
y_15_A310
y_16_A310
y_17_A310
y_18_A310
y_19_A310
y_20_A310
y_21_A310
y_22_A310
y_23_A310
y_24_A310
y_25_A310
y_26_A310
y_27_A310
y_28_A310
y_29_A310
y_30_A310
y_31_A310
y_32_A310
y_0_A320
y_1_A320
y_2_A320
y_3_A320
y_4_A320
y_5_A320
y_6_A320
y_7_A320
y_8_A320
y_9_A320
y_10_A320
y_11_A320
y_12_A320
y_13_A320
y_14_A320
y_15_A320
y_16_A320
y_17_A320
y_18_A320
y_19_A320
y_20_A320
y_21_A320
y_22_A320
y_23_A320
y_24_A320
y_25_A320
y_26_A320
y_27_A320
y_28_A320
y_29_A320
y_30_A320
y_31_A320


In [239]:
for k in ac_list:
    for n in airports:
        for i in nodes[k][n]:
            print(f[l,k].varName)



KeyError: 8

In [19]:
for v in m.getVars():
    print(v.varName)
for c in m.getConstrs():
    print(c.ConstrName)

f_S4120_A310
f_S4120_A320
f_S4220_A310
f_S4220_A320
f_S4124_A310
f_S4124_A320
f_S4128_A310
f_S4128_A320
f_S4320_A310
f_S4320_A320
f_S4212_A310
f_S4212_A320
f_S4160_A310
f_S4160_A320
f_S4222_A310
f_S4222_A320
f_S4221_A310
f_S4221_A320
f_S4121_A310
f_S4121_A320
f_S4229_A310
f_S4229_A320
f_S4125_A310
f_S4125_A320
f_S4129_A310
f_S4129_A320
f_S4321_A310
f_S4321_A320
f_S4213_A310
f_S4213_A320
f_S4161_A310
f_S4161_A320
f_S4223_A310
f_S4223_A320
f_S4230_A310
f_S4230_A320
y_ak[A310,A310]
y_ak[A310,A320]
y_ak[A320,A310]
y_ak[A320,A320]
one_ac[S4120]
one_ac[S4220]
one_ac[S4124]
one_ac[S4128]
one_ac[S4320]
one_ac[S4212]
one_ac[S4160]
one_ac[S4222]
one_ac[S4221]
one_ac[S4121]
one_ac[S4229]
one_ac[S4125]
one_ac[S4129]
one_ac[S4321]
one_ac[S4213]
one_ac[S4161]
one_ac[S4223]
one_ac[S4230]
ac_balance[A310,BOS]
ac_balance[A310,FNC]
ac_balance[A310,OPO]
ac_balance[A310,PDL]
ac_balance[A310,LIS]
ac_balance[A310,YTO]
ac_balance[A320,BOS]
ac_balance[A320,FNC]
ac_balance[A320,OPO]
ac_balance[A320,PDL]
ac_bal